In [1]:
# Importing necessary libraries
import scrapy
from scrapy.http import TextResponse
import requests
import pandas as pd

# Grabbing over all 1000 book records over 20 pages

In [2]:
# Using Scrapy library for grabbing the title,rating,price upc,tax,availabilty,reviews of the books from all pages.

page_still_valid=True
url='http://books.toscrape.com/'

#Creating a dictionary so we can convert text rating into integer
dict1={'One':1,'Two':2,'Three':3,'Four':4,'Five':5}

#Creating empty list in order to append the scraped data into their respective lists
title=[]
upc=[]
product_type=[]
price_exctax=[]
price_inctax=[]
tax=[]
availability=[]
no_of_reviews=[]
rating=[]
price=[]

# With the help of While loops we can scan through every page and grab whatever we need.
while page_still_valid:
    #Putting requests for the first page
    r=requests.get(url)
    response=TextResponse(r.url,body=r.text,encoding="utf-8")
    for x in response.css('h3 a::attr(title)').getall():
        title.append(x)
    for x in response.css('article.product_pod'):
        rating.append(f"{dict1[x.css('p')[0].attrib['class'].split()[1]]}/5")
        price.append(x.css('p.price_color')[0].get().split('>')[1][1:-3])
        
    #Using for loops to get access to each product url so we can grab extra information
    for x in response.css('div.image_container a::attr(href)'):
        if 'catalogue/' in x.get():
            r2=requests.get('http://books.toscrape.com/'+x.get())
        else:
            r2=requests.get('http://books.toscrape.com/catalogue/'+x.get())
        response2=TextResponse(r2.url,body=r2.text,encoding="utf-8")
        #Creating an empty list in which we'll append the extra grabbed data apart from title,price,rating
        a=[]
        for x in response2.css('td'):
            a.append(x.get()[4:-5])
        upc.append(a[0])
        product_type.append(a[1])
        price_exctax.append(a[2][1::])
        price_inctax.append(a[3][1::])
        tax.append(a[4][1::])
        availability.append(a[5].split()[2][1::])
        no_of_reviews.append(a[6])
    #Grabbing the url of the next page as were under while loops so it will keep performing the loop
    next_page=response.css('li.next a::attr(href)').get()
    #Checking if the next page link is there or not,if not then to break the loop as we must on the last page
    if next_page is None:
        break
    elif 'catalogue/' in next_page:
        url='http://books.toscrape.com/'+next_page
    else:
        url="http://books.toscrape.com/catalogue/"+next_page
    

In [3]:
#Checking if the length of every list is same so that we don't face any error while creating a DataFrame using the above appended list
print(len(title),
len(upc),
len(product_type),
len(price_exctax),
len(price_inctax),
len(tax),
len(availability),
len(no_of_reviews),
len(rating),
len(price))

1000 1000 1000 1000 1000 1000 1000 1000 1000 1000


In [4]:
#Creating a DataFrame using Pandas library and adding all the columns in it
df=pd.DataFrame(title).rename(columns={0:'Title'})
df['price']=price
df['rating']=rating
df['product_type']=product_type
df['price_excluding_tax']=price_exctax
df['price_including_tax']=price_inctax
df['tax']=tax
df['upc']=upc
df['no_of_reviews']=no_of_reviews
df['availability']=availability

# Books DataFrame

In [5]:
df

,Title,price,rating,product_type,price_excluding_tax,price_including_tax,tax,upc,no_of_reviews,availability
0,A Light in the Attic,£51.77,3/5,Books,£51.77,£51.77,£0.00,a897fe39b1053632,0,22
1,Tipping the Velvet,£53.74,1/5,Books,£53.74,£53.74,£0.00,90fa61229261140a,0,20
2,Soumission,£50.10,1/5,Books,£50.10,£50.10,£0.00,6957f44c3847a760,0,20
3,Sharp Objects,£47.82,4/5,Books,£47.82,£47.82,£0.00,e00eb4fd7b871a48,0,20
4,Sapiens: A Brief History of Humankind,£54.23,5/5,Books,£54.23,£54.23,£0.00,4165285e1663650f,0,20
...,...,...,...,...,...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,1/5,Books,£55.53,£55.53,£0.00,cd2a2a70dd5d176d,0,1
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,4/5,Books,£57.06,£57.06,£0.00,bfd5e1701c862ac3,0,1
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,5/5,Books,£16.97,£16.97,£0.00,19fec36a1dfb4c16,0,1
998,1st to Die (Women's Murder Club #1),£53.98,1/5,Books,£53.98,£53.98,£0.00,f684a82adc49f011,0,1


# Images of books on the first page

In [6]:
url='http://books.toscrape.com/'
r=requests.get(url)
response=TextResponse(r.url,body=r.text,encoding='utf-8')

In [15]:
#Using for loops to go over list of images
#Had to comment it out as it was just links as output which I copied and pasted below for output as you can see
#for x in response.css('div.image_container img').xpath('@src'):
    #print(f'<img src="{url}{x.get()}">')

<img src="http://books.toscrape.com/media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg">
<img src="http://books.toscrape.com/media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg">
<img src="http://books.toscrape.com/media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg">
<img src="http://books.toscrape.com/media/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg">
<img src="http://books.toscrape.com/media/cache/be/a5/bea5697f2534a2f86a3ef27b5a8c12a6.jpg">
<img src="http://books.toscrape.com/media/cache/68/33/68339b4c9bc034267e1da611ab3b34f8.jpg">
<img src="http://books.toscrape.com/media/cache/92/27/92274a95b7c251fea59a2b8a78275ab4.jpg">
<img src="http://books.toscrape.com/media/cache/3d/54/3d54940e57e662c4dd1f3ff00c78cc64.jpg">
<img src="http://books.toscrape.com/media/cache/66/88/66883b91f6804b2323c8369331cb7dd1.jpg">
<img src="http://books.toscrape.com/media/cache/58/46/5846057e28022268153beff6d352b06c.jpg">
<img src="http://books.toscrape.com/media/cache/be/f4/bef44da28c98f905a3ebec0b87be8530.jpg">
<img src="http://books.toscrape.com/media/cache/10/48/1048f63d3b5061cd2f424d20b3f9b666.jpg">
<img src="http://books.toscrape.com/media/cache/5b/88/5b88c52633f53cacf162c15f4f823153.jpg">
<img src="http://books.toscrape.com/media/cache/94/b1/94b1b8b244bce9677c2f29ccc890d4d2.jpg">
<img src="http://books.toscrape.com/media/cache/81/c4/81c4a973364e17d01f217e1188253d5e.jpg">
<img src="http://books.toscrape.com/media/cache/54/60/54607fe8945897cdcced0044103b10b6.jpg">
<img src="http://books.toscrape.com/media/cache/55/33/553310a7162dfbc2c6d19a84da0df9e1.jpg">
<img src="http://books.toscrape.com/media/cache/09/a3/09a3aef48557576e1a85ba7efea8ecb7.jpg">
<img src="http://books.toscrape.com/media/cache/0b/bc/0bbcd0a6f4bcd81ccb1049a52736406e.jpg">
<img src="http://books.toscrape.com/media/cache/27/a5/27a53d0bb95bdd88288eaf66c9230d7e.jpg">
​
